# Error Analysis

In this notebook we want to get an impression wether there are some patterns in the data that was misclassified by our best XGBoost model (see General.ipynb).

In [ ]:
import pickle
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In order to see patterns from the predictions, a column indicating wether the prediction of a datapoint was a true positive, a true negative, a false positive or a false negative was created. The results were visualised in a pairplot. 

In [ ]:
# Load test data
y_test = pd.read_csv('data/y_test.csv')
X_test = pd.read_csv('data/unscaled_Xtest.csv')

# Load best model
best_model = pickle.load(open('./models/best_xgb_model', 'rb'))

# Predict on best model
y_pred = best_model.predict(X_test)

# Reshape test labels to 1D array
y_test = np.array(y_test)
y_test = np.squeeze(y_test)

# Array of True and False marking correct/incorrect predictions
corr_pred = y_test == y_pred

X_test_analysis = pd.DataFrame(X_test)
X_test_analysis['corr_pred'] = corr_pred
X_test_analysis['true_res'] = y_test


# Identifying wether prediction is True Positive (TP), True Negative (TN), False Positive (FP) or False Negative (FN)
result_category = np.empty(len(y_test), dtype=object)
mtp = (corr_pred == True)
mfp = (corr_pred == False)
mtt = (y_test == 1)
mft = (y_test == 0)

mTP = np.logical_and(mtp, mtt)
mFP = np.logical_and(mtp, mft)
mTN = np.logical_and(mfp, mft)
mFN = np.logical_and(mfp, mtt)

result_category[mTP] = 'TP'
result_category[mFP] = 'FP'
result_category[mTN] = 'TN'
result_category[mFN] = 'FN'

X_test_analysis['result_category'] = result_category


In [ ]:
sns.pairplot(X_test_analysis.drop(['corr_pred', 'true_res'],axis=1), hue='result_category')
plt.savefig('images/Error_Analysis/pairplot_predicted_results.png', dpi=600)

Overall there is no pattern visible indicating why the model failed on specific datapoints. However, regarding the false positives, it becomes clear, that some statements concluded from the EDA should be taken into consideration, next to what the model predicted. This includes 

- Too high goals increase the risk of a failed campaign 
- Campaigns launched in the summer months (especially July) have a higher risk of failing then campaigns launched in other months 
- Campaigns with a duration of approximately 60 days (2 month) have a higher risk of failing 
